# **EfficientDet Inference**

* [Dependencies and imports](#section-one)
* [Overide functions](#section-two)
* [Basic configurations](#section-three)
* [Prepare Dataset](#section-four)
    * [DICOM files](#section-four-one)
    * [Augmentation for resizing image](#section-four-two)
* [Create custom dataset](#section-five)
* [Object detection and multi-class classification](#section-seven)
    * [Load pre-trained models](#section-seven-one)
    * [Ensemble models](#section-seven-two)
    * [Create test dataset and dataloader](#section-seven-three)
    * [Get predictions](#section-seven-four)
* [Create submission file](#section-eight)

<a id="section-one"></a>
## **Dependencies and imports**

In [ ]:
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
!conda install '../input/dicom/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '../input/dicom/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '../input/dicom/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '../input/dicom/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '../input/dicom/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '../input/dicom/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
import sys
sys.path.append('../input/timm-efficientdet-pytorch')
sys.path.append("../input/omegaconf")
sys.path.append("../input/weightedboxesfusion")
sys.path.append('../input/imagemodels/pytorch-image-models-master')

import torch
from torch import nn
import timm
import numpy as np
import pandas as pd
import gc
import os
import ast
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
# --- effdet ---
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet.efficientdet import HeadNet
# --- ensemble boxes ---
import ensemble_boxes
from ensemble_boxes import *
# --- data ---
from torch.utils.data import Dataset,DataLoader
# --- images ---
import albumentations as A
import cv2
# --- wandb ---
import wandb
from kaggle_secrets import UserSecretsClient
# --- dicom ---
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
OFFLINE = True

if not OFFLINE:
    user_secrets = UserSecretsClient()
    wandb_key = user_secrets.get_secret("wandb-key")
    wandb.login(key=wandb_key)

    run = wandb.init(project="siim-covid19-detection", name="inference", mode='online')

<a id="section-two"></a>
## **Overide functions**
_post_process in effdet/bench.py raised error due division for calculating indices returned float (using '/') instead of an int (using '//')

In [ ]:
import effdet.bench as bench
from effdet.anchors import MAX_DETECTION_POINTS

def new_post_process(config, cls_outputs, box_outputs):
    """Selects top-k predictions.

    Post-proc code adapted from Tensorflow version at: https://github.com/google/automl/tree/master/efficientdet
    and optimized for PyTorch.

    Args:
        config: a parameter dictionary that includes `min_level`, `max_level`,  `batch_size`, and `num_classes`.

        cls_outputs: an OrderDict with keys representing levels and values
            representing logits in [batch_size, height, width, num_anchors].

        box_outputs: an OrderDict with keys representing levels and values
            representing box regression targets in [batch_size, height, width, num_anchors * 4].
    """
    batch_size = cls_outputs[0].shape[0]
    cls_outputs_all = torch.cat([
        cls_outputs[level].permute(0, 2, 3, 1).reshape([batch_size, -1, config.num_classes])
        for level in range(config.num_levels)], 1)

    box_outputs_all = torch.cat([
        box_outputs[level].permute(0, 2, 3, 1).reshape([batch_size, -1, 4])
        for level in range(config.num_levels)], 1)

    _, cls_topk_indices_all = torch.topk(cls_outputs_all.reshape(batch_size, -1), dim=1, k=MAX_DETECTION_POINTS)
    ############################# changed / to // as indices should be int64 #############################
    indices_all = cls_topk_indices_all // config.num_classes 
    ######################################################################################################
    classes_all = cls_topk_indices_all % config.num_classes

    box_outputs_all_after_topk = torch.gather(
        box_outputs_all, 1, indices_all.unsqueeze(2).expand(-1, -1, 4))

    cls_outputs_all_after_topk = torch.gather(
        cls_outputs_all, 1, indices_all.unsqueeze(2).expand(-1, -1, config.num_classes))
    cls_outputs_all_after_topk = torch.gather(
        cls_outputs_all_after_topk, 2, classes_all.unsqueeze(2))

    return cls_outputs_all_after_topk, box_outputs_all_after_topk, indices_all, classes_all

bench._post_process  = new_post_process

<a id="section-three"></a>
## **Basic configurations**

In [ ]:
NONE = 'none'
OPACITY = 'opacity'

NEGATIVE = 'negative'
TYPICAL = 'typical'
INDERTEMINATE = 'indeterminate'
ATYPICAL = 'atypical'

class Configs:
    n_folds = 5
    img_size = 512
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    batch_size = 4
    num_workers = 8
    study_level = {1:TYPICAL, 2:INDERTEMINATE, 3:ATYPICAL}

<a id="section-four"></a>
## **Prepare Dataset**

<a id="section-four-one"></a>
#### **DICOM files**

**Get DICOM files paths**

In [ ]:
paths = []
for dirname, _, filenames in os.walk('../input/siim-covid19-detection/test'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))

In [ ]:
def get_img_id(path):
    # dicom path of format '../input/siim-covid19-detection/test/study_id/dir/image_id.dcm'
    return path.split('/')[-1].split('.')[0] # extract img_id from path

def get_study_id(path):
    # dicom path of format '../input/siim-covid19-detection/test/study_id/dir/dicom_image'
    return path.split('/')[-3]

**Get dicom image**

In [ ]:
def get_dicom_img(path):
    data_file = pydicom.dcmread(path)
    img = apply_voi_lut(data_file.pixel_array, data_file)

    if data_file.PhotometricInterpretation == "MONOCHROME1":
        img = np.amax(img) - img
    
    # Rescaling grey scale between 0-255 and convert to uint
    img = img - np.min(img)
    img = img / np.max(img)
    img = (img * 255).astype(np.uint8)

    return img

<a id="section-four-two"></a>
#### **Augmentation for resizing image**

In [ ]:
def get_test_transforms():
    return A.Compose([A.Resize(height=Configs.img_size, width=Configs.img_size, p=1.0),], p=1.0)

<a id="section-five"></a>
## **Create custom dataset**

In [ ]:
class Covid19TestDataset(Dataset):
    def __init__(self, dicom_paths, transform=None):
        super().__init__()
        self.paths = dicom_paths
        self.transform = transform

    def __getitem__(self, idx):
        img_id = get_img_id(self.paths[idx])
        study_id = get_study_id(self.paths[idx])
        img = get_dicom_img(self.paths[idx])
        #img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        
        if self.transform:
            transformed = self.transform(image=img)
            img = transformed['image']
           
        # normalize img
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img /= 255.0
        
        # convert image into a torch.Tensor
        img = torch.as_tensor(img, dtype=torch.float32)
        #idx = torch.tensor([idx])
        
        # permute image to [C,H,W] from [H,W,C] and normalize
        img = img.permute(2, 0, 1)
        
        return img, img_id, study_id
    
    def __len__(self):
        return len(self.paths)

<a id="section-seven"></a>
## **Object detection and multi-class classification**

<a id="section-seven-one"></a>
#### **Load pre-trained models**

In [ ]:
# get efficientdet model
def get_model(model_name):
    config = get_efficientdet_config(model_name)
    model = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = len(Configs.study_level)
    config.image_size = Configs.img_size

    checkpoint = torch.load('../input/efficientdet/efficientdet_d7-f05bf714.pth')
    model.load_state_dict(checkpoint)
    
    model.class_net = HeadNet(config, num_outputs=config.num_classes)

    model = DetBenchEval(model, config)
    model.eval();
    return model

# load checkpoint
def load_obj_detection_model(model_name, fold, checkpoint_path):
    model = get_model(model_name)
    checkpoint = torch.load(checkpoint_path)['model_state_dict']
    checkpoint['anchors.boxes'] = checkpoint.pop('anchor_labeler.anchors.boxes')
    model.load_state_dict(checkpoint)
    
    return model.to(Configs.device)

models = []
model_name = 'tf_efficientdet_d7'
for fold in range(Configs.n_folds): 
    models.append(load_obj_detection_model(model_name, fold, f'../input/models/object_detection_models/{model_name}/{model_name}_fold{fold}/best-checkpoint.bin'))

In [ ]:
def log_prediction(image, image_id, boxes, labels):
    new_image = image.permute(1, 2, 0).numpy().copy() 
    new_image = (new_image*255).astype(np.uint8)
    image_size = max(image.shape)
    caption = []
    
    for label in labels:
        caption.append(Configs.study_level[label])
    for box in boxes:
        x, y, w, h = box
        cv2.rectangle(new_image, (int(x), (int(y))), (int(x+w),  int(y+h)), (255,0,0), image_size//200)
        
    wandb.log({'predictions/'+image_id: [wandb.Image(new_image, caption=', '.join(caption))]})

<a id="section-seven-two"></a>
#### **Ensemble models**

In [ ]:
# wbf = weighted boxes fusion for ensembling boxes from object detection models
def run_wbf(predictions, img_idx, iou_thr=0.65, skip_box_thr=0.4, weights=None):
    # prediction[img_idx] is the prediction of each model for the same image (the image in img_idx)
    # weighted_boxes_fusion function received boxes with values in range [0-1]
    # normalize by dividing over Configs.img_size-1 (includes 0, therefore -1)
    boxes = [(prediction[img_idx]['boxes']/(Configs.img_size-1)).tolist() for prediction in predictions]
    scores = [prediction[img_idx]['scores'].tolist() for prediction in predictions]
    labels = [prediction[img_idx]['labels'].astype(int).tolist() for prediction in predictions]
    boxes, scores, labels = ensemble_boxes.ensemble_boxes_wbf.weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    # scale back boxes by multiplying with Configs.img_size-1
    boxes = boxes*(Configs.img_size-1)
    return boxes, scores, labels

# prediction for a single batch
def batch_predictions(images, image_ids, score_threshold=0.4):
    with torch.no_grad():
        predictions = []
        images = images.float().cuda()
        
        # for each model get predictions for each image in batch
        for model in models:  
            model_predictions = []
            outputs = model(images, torch.tensor([1]*images.shape[0]).float().cuda())
        
            for i, (image, image_id) in enumerate(zip(images, image_ids)):
                boxes = outputs[i].detach().cpu().numpy()[:,:4]    
                scores = outputs[i].detach().cpu().numpy()[:,4]
                labels = outputs[i].detach().cpu().numpy()[:, 5]
                indexes = np.where(scores > score_threshold)[0]
                
                model_predictions.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
                'labels': labels[indexes]
                })
            # append model prediction to all predictions
            predictions.append(model_predictions)     
            
    return predictions

def get_boxes(image, boxes):
    # get ratio to scale boxes
    new_image = image.permute(1, 2, 0).numpy().copy() 
    height, width = new_image.shape[0], new_image.shape[1]
    h_ratio = height / Configs.img_size
    w_ratio = width / Configs.img_size
    
    # scale and convert from xywh to xyxy
    new_boxes = []
    for box in boxes:
        x,y,w,h = box
        x1=x*w_ratio
        x2=(x+w)*w_ratio
        y1=y*h_ratio
        y2=(y+h)*h_ratio
        new_boxes.append(np.array([int(x1),int(y1),int(x2),int(y2)]))
        
    return np.array(new_boxes)    
            
def ensemble_predictions(test_loader):
    # returns a df with img_id, study_id and predicted values (labels, scores and boxes) for each image
    img_ids = []
    stdy_ids = []
    pred_boxes =[]
    pred_scores = []
    pred_labels = []
    
    for images, image_ids, study_ids in tqdm(test_loader):
        # get batch predictions of all models
        # predictions[0] = batch predictions for model0, predictions[1] = batch predictions for model1 and etc...
        predictions = batch_predictions(images, image_ids)
        for i, (image, image_id, study_id) in enumerate(zip(images, image_ids, study_ids)):
            # for each image get ensembled prediction by using wbf
            boxes, scores, labels = run_wbf(predictions, img_idx=i)
            if not OFFLINE:
                log_prediction(image, image_id, boxes, labels)
            new_boxes = get_boxes(image, boxes)
            
            img_ids.append(image_id)
            stdy_ids.append(study_id)
            pred_boxes.append(new_boxes)
            pred_scores.append(scores)
            pred_labels.append(labels.astype(int))
            
    return pd.DataFrame({'img_id':img_ids, 'study_id':stdy_ids, 'labels':pred_labels, 'scores':pred_scores, 'boxes':pred_boxes})

<a id="section-seven-three"></a>
#### **Create test dataset and dataloader**

In [ ]:
test_dataset = Covid19TestDataset(paths, transform=get_test_transforms())

In [ ]:
from torch.utils.data.sampler import SequentialSampler

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=Configs.batch_size,
    num_workers=Configs.num_workers,
    shuffle=False,
    pin_memory=False,
    drop_last=False,
)

<a id="section-seven-four"></a>
#### **Get predictions**

In [ ]:
preds_df = ensemble_predictions(test_loader)

In [ ]:
preds_df.head()

<a id="section-eight"></a>
## **Create submission file**

In [ ]:
def create_image_prediction_string(row):
    pred_strings = []
    if len(row['labels'].values[0]) > 0:
        # if there are predicted labels
        for score,box in zip(row['scores'].values[0], row['boxes'].values[0]):
            x1,y1,x2,y2 = box
            pred_strings.append("opacity {:.1f} {} {} {} {}".format(score, x1, y1, x2, y2))
        return ' '.join(pred_strings)
    else:
        return 'none 1 0 0 1 1'

def create_study_prediction_string(rows):
    pred_strings = []
    for index, row in rows.iterrows():
        if len(row['labels'])==0:
            pred_strings.append('negative 1 0 0 1 1')
        else:
            labels = []
            for label in row['labels']:
                if label not in labels:
                    labels.append(label)
            for label in labels:
                pred_strings.append("{} 1 0 0 1 1".format(Configs.study_level[label]))
    return ' '.join(pred_strings)

In [ ]:
def create_prediction_string(sample_submission):
    ids = []
    preds = []
    
    for index, row in sample_submission.iterrows():
        id = row['id']
        ids.append(id)
        if id.endswith('_image'):
            id = id.split('_')[0]
            preds.append(create_image_prediction_string(preds_df[preds_df['img_id'] == id]))
        else:
            id = id.split('_')[0]
            preds.append(create_study_prediction_string(preds_df[preds_df['study_id'] == id]))
    
    return pd.DataFrame({'id':ids, 'PredictionString':preds})

submission = create_prediction_string(pd.read_csv('../input/siim-covid19-detection/sample_submission.csv'))
submission

In [ ]:
submission.to_csv('./submission.csv', index=False)